In [ ]:
##!/usr/bin/env python
"""plot_point_time_series.py

Script plots the point time series against Flux Tower data to evaluate the WRF-LIS-CABLE physics ensemble

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Thu May  9 12:04:37 AEST 2019

"""

In [1]:
# Load packages

#from __future__ import division
import numpy as np
import math
import netCDF4 as nc
import sys
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray
from mpl_toolkits.basemap import Basemap
from matplotlib.collections import LineCollection
import common_functions as cf
import datetime as dt

In [2]:
# All Simulation Details

sims=["PHYS_TEST_RA1_PBL1_CU1", "PHYS_TEST_RA1_PBL1_CU16", "PHYS_TEST_RA1_PBL1_CU2", "PHYS_TEST_RA1_PBL1_CU3", 
      "PHYS_TEST_RA1_PBL2_CU1", "PHYS_TEST_RA1_PBL2_CU16", "PHYS_TEST_RA1_PBL2_CU2", "PHYS_TEST_RA1_PBL2_CU3", 
      "PHYS_TEST_RA1_PBL5_CU1", "PHYS_TEST_RA1_PBL5_CU16", "PHYS_TEST_RA1_PBL5_CU2", "PHYS_TEST_RA1_PBL5_CU3", 
#      "PHYS_TEST_RA3_PBL2_CU1", "PHYS_TEST_RA3_PBL2_CU16", "PHYS_TEST_RA3_PBL2_CU2", 
      "PHYS_TEST_RA4_PBL1_CU1", "PHYS_TEST_RA4_PBL1_CU16", "PHYS_TEST_RA4_PBL1_CU2", "PHYS_TEST_RA4_PBL1_CU3", 
      "PHYS_TEST_RA4_PBL2_CU1", "PHYS_TEST_RA4_PBL2_CU16", "PHYS_TEST_RA4_PBL2_CU2", "PHYS_TEST_RA4_PBL2_CU3", 
      "PHYS_TEST_RA4_PBL5_CU1", "PHYS_TEST_RA4_PBL5_CU16", "PHYS_TEST_RA4_PBL5_CU2", "PHYS_TEST_RA4_PBL5_CU3", 
      "PHYS_TEST_RA5_PBL1_CU1", "PHYS_TEST_RA5_PBL1_CU16", "PHYS_TEST_RA5_PBL1_CU2", "PHYS_TEST_RA5_PBL1_CU3", 
      "PHYS_TEST_RA5_PBL2_CU1", "PHYS_TEST_RA5_PBL2_CU16", "PHYS_TEST_RA5_PBL2_CU2", "PHYS_TEST_RA5_PBL2_CU3", 
      "PHYS_TEST_RA5_PBL5_CU1", "PHYS_TEST_RA5_PBL5_CU16", "PHYS_TEST_RA5_PBL5_CU2", "PHYS_TEST_RA5_PBL5_CU3"]
rlabels=["Dudhia / YSU / KF","Dudhia / YSU / NTiedtke","Dudhia / YSU / BMJ","Dudhia / YSU / GF",
         "Dudhia / MYJ / KF","Dudhia / MYJ / NTiedtke","Dudhia / MYJ / BMJ","Dudhia / MYJ / GF",
         "Dudhia / MYNN / KF","Dudhia / MYNN / NTiedtke","Dudhia / MYNN / BMJ","Dudhia / MYNN / GF",
#         "CAM / MYJ / KF","CAM / MYJ / NTiedtke","CAM / MYJ / BMJ",
         "RRTMG / YSU / KF","RRTMG / YSU / NTiedtke","RRTMG / YSU / BMJ","RRTMG / YSU / GF",
         "RRTMG / MYJ / KF","RRTMG / MYJ / NTiedtke","RRTMG / MYJ / BMJ","RRTMG / MYJ / GF",
         "RRTMG / MYNN / KF","RRTMG / MYNN / NTiedtke","RRTMG / MYNN / BMJ  ","RRTMG / MYNN / GF",
         "NGoddard / YSU / KF","NGoddard / YSU / NTiedtke","NGoddard / YSU / BMJ  ","NGoddard / YSU / GF",
         "NGoddard / MYJ / KF","NGoddard / MYJ / NTiedtke","NGoddard / MYJ / BMJ  ","NGoddard / MYJ / GF",
         "NGoddard / MYNN / KF","NGoddard / MYNN / NTiedtke","NGoddard / MYNN / BMJ  ","NGoddard / MYNN / GF"]
group="ALL"

#tind = [3,7,11,15,19,23,27,31,35] # For GF
#tind = [4,8,17,18,32] # For Tropics
#tind = [17,18,21,23,28] # For SEA
tind = [4,17,18,21,32] # for Total


In [3]:
#llabels = [rlabels[3],rlabels[7],rlabels[11],rlabels[15],rlabels[19],rlabels[23],rlabels[27],rlabels[31],rlabels[35]]
#llabels = [rlabels[4],rlabels[8],rlabels[17],rlabels[18],rlabels[32]]
#llabels = [rlabels[17],rlabels[18],rlabels[21],rlabels[23],rlabels[28]]
llabels = [rlabels[4],rlabels[17],rlabels[18],rlabels[21],rlabels[32]]


In [4]:
# Define USER input arguments

# Year / Period of interest
syear = np.int('2008')
eyear = np.int('2010')
nlat = 152
nlon = 223

# Data directory 
datadir='/g/data/hh5/tmp/WRF-CABLE/AUS44/postprocess/'

nmod = len(sims)
obs="AWAP"

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
fig_name_prefix='OZFLUX_TIME_SERIES_'

if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)

# Landsea mask
mask_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/geo_em.d01.nc'
lis_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/bdy_data/lis_input.d01.nc'

# Variables
vlabels=['$Q_{E}$ [W $m^{-2}$]']
nvar=len(vlabels)
mx = [250.]
mn = [0.]
mnmx = [50.]
lspace = 10


In [5]:
# Define Flux Towers

fluxdir='/g/data3/w97/azh561/Ozflux/'
fluxsites=['AU-DaP_2008-2012_OzFlux_Flux.nc','AU-DaS_2008-2016_OzFlux_Flux.nc',
           'AU-Dry_2009-2016_OzFlux_Flux.nc','AU-How_2002-2016_OzFlux_Flux.nc','AU-Otw_2008-2010_OzFlux_Flux.nc',
           'AU-Stp_2009-2016_OzFlux_Flux.nc','AU-Tum_2002-2016_OzFlux_Flux.nc']

#variables to read in: time, Qle, Qle_qc, Qh, Qh_qc, LWup, LWup_qc, SWup, SWup_qc, latitude,longitude
# Coast? 'AU-Cow_2009-2015_OzFlux_Flux.nc'
#do Qle only initially

In [6]:
# Load the mask file
filemask    = nc.Dataset(lis_file)
lsmask      = filemask.variables['LANDMASK'][:]
lsmask      = lsmask.astype(int)  
lat2d       = filemask.variables['lat'][:]
lon2d       = filemask.variables['lon'][:]
filemask.close()

# If the longitude spans [-180 180] then update to that instead it is [0 360] - better of AUS domain
#if np.min(lon) < 0.0:
#    print('Longitude values span [-180 180] will change to [0 360]')
#    for jj in range(len(lon)):
#        if lon[jj] < 0.0:
#            lon[jj]=lon[jj]+360

#latT = lat[(lat>=latN) & (lat<=latX)]
#lonT = lon[(lon>=lonN) & (lon<=lonX)]

In [7]:
# Function to plot data
def plot_ts(time,tsdata,rlabels,vlabels,mtitle,figurename,mx,mn,mnmx,lspace):
    """This function plots time series for observations and models"""

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    import string
    
    # Figure formatting
    plt.rcParams['savefig.dpi']=500
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"

    # Define dimensions
    nmod = tsdata.shape[0] - 3 # As obs, mx and mn are included
    nvar = tsdata.shape[1]
    nt = tsdata.shape[2]
      
    # Create figure object and subplots
#    fig, axarr = plt.subplots(nvar, 1, figsize=(20.0,5.0*(nvar)), squeeze=False)
    fig, axarr = plt.subplots(nvar, 1, figsize=(20.0,10.0), squeeze=False)
    tarr = np.arange(0,nt)
    
    # Have a look at the colormaps here and decide which one you'd like:
    # http://matplotlib.org/1.2.1/examples/pylab_examples/show_colormaps.html
    #https://stackoverflow.com/questions/4805048/how-to-get-different-colored-lines-for-different-plots-in-a-single-figure/4805456#4805456
    # Would be nice to colorcode according to skill...e.g. darker time series == bad
    colors = ["red","blue","green","orange","purple"] 
    
    # Iterate through variables
    for vind in range(nvar):

        # Models
        axarr[vind,0].fill_between(tarr, tsdata[1,vind,:], tsdata[2,vind,:], alpha=0.25, facecolor='grey')
        for mind in range(nmod):
            axarr[vind,0].plot(tarr,tsdata[mind+3,vind,:], linewidth=4,color=colors[mind], linestyle='-', label=rlabels[mind])

        # Observations
        axarr[vind,0].plot(tarr,tsdata[0,vind,:],linestyle='-',linewidth=4,color='black',label='Observed')
              
        # Fix Labelling
        axarr[vind,0].set_ylabel('%s' %(vlabels[vind]), fontweight = 'bold',fontsize=20)
        axarr[vind,0].set_ylim(mn[vind],mx[vind])
        axarr[vind,0].set_yticks(np.arange(mn[vind],mx[vind],mnmx[vind]))
        axarr[vind,0].set_yticklabels(np.arange(np.int(mn[vind]),np.int(mx[vind]),np.int(mnmx[vind])), fontsize=18)

        if vind < nvar-1:
            axarr[vind,0].set_xticks([],[])
        else:
            axarr[vind,0].set_xticks(tarr[::lspace])
            axarr[vind,0].set_xticklabels(time[::lspace],rotation=90,fontsize=18)

    axarr[0,0].set_title(mtitle, fontweight = 'bold')
#    legend = axarr[-1,0].legend(loc='upper center', bbox_to_anchor=(0.5,-0.375), ncol=3, fontsize=20)
    legend = axarr[-1,0].legend(loc='upper center', bbox_to_anchor=(0.5,-0.8), ncol=3, fontsize=20)

    fig.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(figurename,bbox_extra_artists=(legend,), bbox_inches='tight')
    plt.close(fig)


In [8]:
for rind in range(len(fluxsites)):
    
    # Read in the Flux Tower data
    filename='%s/daily_%s' %(fluxdir,fluxsites[rind])
    file = nc.Dataset(filename)
    obshfls = file.variables['Qle'][:]
    obstime = file.variables['time'][:]
    obstimeunits = file.variables['time'].units
    obsbasetime = obstimeunits.split(" ")[2]
    by,bm,bd = obsbasetime.split("-")
    obslat = file.variables['latitude'][0][0]
    obslon = file.variables['longitude'][0][0]
    Elabel = file.site_name
    
    # Calculate time array of all times in dataset
    oyears = np.empty((len(obstime)),dtype=np.int)
    for tt in range(len(obstime)):
        tmp = dt.datetime(int(by),int(bm),int(bd)) + dt.timedelta(seconds=int(obstime[tt]))
        oyears[tt] = tmp.year
        del tmp

    # Retrieve data corresponding to 2009
    oind = np.where(oyears==2009)[0]
    tsall = np.empty((nmod+1,365),dtype=np.float64)
    tsall[0,:] = obshfls[oind][:,0,0]
    
    # Get lat/lon corresponding to the flux site
    # https://stackoverflow.com/questions/28006077/find-i-j-location-of-closest-long-lat-values-in-a-2d-array
    a = abs(lat2d-obslat)+abs(lon2d-obslon)
    i0,j0 = np.unravel_index(a.argmin(),a.shape)
    
    # Loop through the experiments
    for mind,mname in enumerate(sims):
        
        if mind == 0:
            # WRF time
            filename='%s/%s_WRFext_Temps_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
            file = nc.Dataset(filename)
            time = nc.chartostring(file.variables['Times'][:,:])
            timesplit = np.asarray([time[x].split("_")[0] for x in range(len(time))])
            myears = np.asarray([timesplit[x].split("-")[0] for x in range(len(time))])
            file.close()

        # Load hfls data
        filename='%s/%s_lisout_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
        file = nc.Dataset(filename)
        tsall[mind+1,:] = file.variables['lis_hfls_mask'][np.where(myears=='2009')[0],i0,j0]
        file.close()

    tsmxmn = np.empty((8,1,365),dtype=np.float64)
    tsmxmn[0,0,:] = tsall[0,:]
    tsmxmn[1,0,:] = np.nanmin(tsall[1:,:],axis=0)
    tsmxmn[2,0,:] = np.nanmax(tsall[1:,:],axis=0)
    tsmxmn[3,0,:] = tsall[np.int(tind[0])+1,:]
    tsmxmn[4,0,:] = tsall[np.int(tind[1])+1,:]
    tsmxmn[5,0,:] = tsall[np.int(tind[2])+1,:]
    tsmxmn[6,0,:] = tsall[np.int(tind[3])+1,:]
    tsmxmn[7,0,:] = tsall[np.int(tind[4])+1,:]

    # Plot the time series
    figurename='%s/%s%s.png' %(fig_dir,fig_name_prefix,Elabel)
    figuretitle='Flux Site: %s Year: 2009' %(Elabel)
    plot_ts(timesplit[np.where(myears=='2009')[0]],tsmxmn,llabels,vlabels,figuretitle,figurename,mx,mn,mnmx,lspace)
        
    del tsmxmn,tsall,time,timesplit,myears
    del obshfls,obstime,obstimeunits,obsbasetime,by,bm,bd,obslat,obslon,Elabel,oyears,oind,a,i0,j0
    